In [1]:
import mysql.connector
import pandas as pd
import numpy as np

In [2]:
# Connect to MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="!Riel5726",
    database="loans"
)

cursor = connection.cursor()

# Read the CSV file
file_path = "C:/Users/Riel Pienaar/Downloads/loans.csv"
df = pd.read_csv(file_path)

# Get column names from the CSV
columns = df.columns.tolist()

# Replace NaN values with None (MySQL handles None as NULL)
df = df.replace({np.nan: None})

# Drop the table if it already exists
table_name = "my_table"
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

# Create a new table with the same column names as the CSV
create_table_sql = f"CREATE TABLE {table_name} (id INT AUTO_INCREMENT PRIMARY KEY, " + ", ".join([f"{col} VARCHAR(255)" for col in columns]) + ")"
cursor.execute(create_table_sql)

# Create the SQL INSERT statement dynamically
sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

# Batch size for batch insertion
batch_size = 100  # Modify this if needed

# Insert the data in batches
for i in range(0, len(df), batch_size):
    # Select rows in the current batch
    batch_rows = df.iloc[i:i + batch_size]
    # Convert batch rows to a list of tuples
    values = [tuple(row) for row in batch_rows.to_numpy()]
    # Execute many rows at once
    cursor.executemany(sql, values)

# Commit the transaction
connection.commit()

# Close the connection
cursor.close()
connection.close()

print("File uploaded and table created successfully with batch insertion!")

File uploaded and table created successfully with batch insertion!
